# Installing Necessary Libraries

In [6]:
import sys
!{sys.executable} -m pip install opencv-contrib-python

# Importing Required Libraries

In [7]:
import cv2 
import math

# Loading and Processing Video Files

In [8]:
# List of video files
video_paths = ["Traffic_Laramie_1.mp4", "Traffic_Laramie_2.mp4"]

# Loop through each video and display properties
for video_path in video_paths:
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path}")
        continue  # Skip to the next video if it can't be opened

    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))   # Width of the frames
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) # Height of the frames
    frame_rate = cap.get(cv2.CAP_PROP_FPS)                 # Frames per second
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # Total number of frames
    duration = total_frames / frame_rate if frame_rate > 0 else 0  # Duration in seconds

    # Print video properties
    print(f"\nVideo Properties for {video_path}:")
    print(f"  - Frame Width: {frame_width} pixels")
    print(f"  - Frame Height: {frame_height} pixels")
    print(f"  - Frame Rate: {frame_rate:.2f} FPS")
    print(f"  - Total Frames: {total_frames}")
    print(f"  - Duration: {duration:.2f} seconds")

    cap.release()  # Release video capture before moving to the next


Video Properties for Traffic_Laramie_1.mp4:
  - Frame Width: 1040 pixels
  - Frame Height: 600 pixels
  - Frame Rate: 25.00 FPS
  - Total Frames: 4448
  - Duration: 177.92 seconds

Video Properties for Traffic_Laramie_2.mp4:
  - Frame Width: 1040 pixels
  - Frame Height: 600 pixels
  - Frame Rate: 25.00 FPS
  - Total Frames: 2642
  - Duration: 105.68 seconds


# Function to identify car and background subtraction

In [9]:
def car_counter(video_cap, bg_sub):
    '''
    This function tracks and counts moving cars in a specified area using background subtraction.
    '''

    # Check if the video file was successfully opened
    if not video_cap.isOpened():
        print("Error: Could not open video file.")
        return

    # Get video properties
    amount_of_frames = int(video_cap.get(cv2.CAP_PROP_FRAME_COUNT))  # Total number of frames
    video_fps = video_cap.get(cv2.CAP_PROP_FPS)  # Frames per second
    video_duration = amount_of_frames / video_fps if video_fps > 0 else 1  # Avoid division by zero

    # Initialize variables for tracking
    previous_frame_centroids = []  # Stores centroids from previous frame
    tracked_objects = {}  # Dictionary to track objects
    tracking_id = 0  # Unique ID for each tracked object
    car_counter = 0  # Counter for detected cars

    # Process each frame in the video
    for _ in range(amount_of_frames):
        current_frame_centroids = []  # Stores centroids in the current frame

        # Read a frame from the video
        ret, frame = video_cap.read()
        if not ret or frame is None:
            print("End of video or error reading frame.")
            break

        # Define Region of Interest (ROI) to limit detection area
        detection_area = frame[260:600, 0:1040]  

        # Apply background subtraction to detect moving objects
        fgmask = bg_sub.apply(detection_area)
        _, fgmask = cv2.threshold(fgmask, 254, 255, cv2.THRESH_BINARY)  # Binarize the mask
        contours, _ = cv2.findContours(fgmask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)  # Find object contours

        # Loop through detected contours
        for cnt in contours:
            area = cv2.contourArea(cnt)  # Get contour area
            x, y, w, h = cv2.boundingRect(cnt)  # Get bounding box for detected object

            # Large objects, likely cars, split into two parts for better tracking
            if x < 520 and area > 13500 and h > 130:
                cx1, cy1 = int(x + (x + w) / 2), int(y + (y + h/2) / 2)
                cx2, cy2 = int(x + (x + w) / 2), int(y + h/2 + (y + h/2) / 2)
                current_frame_centroids.extend([(cx1, cy1), (cx2, cy2)])

                # Draw bounding boxes around detected cars
                cv2.rectangle(detection_area, (x, y), (x + w, y + int(h/2)), (0, 255, 0), 1)
                cv2.rectangle(detection_area, (x, y + int(h/2)), (x + w, y + h), (0, 255, 0), 1)

            # Smaller objects, also likely vehicles
            elif area > 4700:
                cx, cy = int(x + (x + w) / 2), int(y + (y + h) / 2)
                current_frame_centroids.append((cx, cy))
                
                # Draw a bounding box around detected objects
                cv2.rectangle(detection_area, (x, y), (x + w, y + h), (0, 255, 0), 1)

        # Track detected objects using centroids
        for ccentroid in current_frame_centroids:
            nearest_track_obj = []  # Store the closest tracked object

            for object_key in tracked_objects:
                tracked_obj = tracked_objects[object_key]
                # Calculate Euclidean distance between tracked and current centroid
                distance = math.sqrt((tracked_obj[0] - ccentroid[0])**2 + (tracked_obj[1] - ccentroid[1])**2)

                # If the object is within a reasonable distance, track it
                if distance < 100:
                    if nearest_track_obj and nearest_track_obj[2] > distance:
                        nearest_track_obj = [ccentroid[0], ccentroid[1], distance, tracked_obj[2]]
                    elif not nearest_track_obj:
                        nearest_track_obj = [ccentroid[0], ccentroid[1], distance, tracked_obj[2]]

            # If no nearby tracked object is found, assign a new tracking ID
            if not nearest_track_obj:
                tracked_objects[tracking_id] = [ccentroid[0], ccentroid[1], tracking_id, True]
                tracking_id += 1
            else:
                update_track_id = nearest_track_obj[3]
                tracked_objects[update_track_id] = [ccentroid[0], ccentroid[1], update_track_id, True]

            # Clear temporary tracking object
            nearest_track_obj.clear()

        # Remove old tracked objects and count cars if they exit the frame
        for object_key in list(tracked_objects.keys()):
            centroid = tracked_objects[object_key]
            if centroid[3]:
                tracked_objects[centroid[2]][3] = False
            else:
                if centroid[0] < 215:  # If the object moves past a certain point
                    car_counter += 1  # Increase car count
                del tracked_objects[centroid[2]]  # Remove old tracked objects

        # Display the car count on the video frame
        cv2.putText(frame, 'Cars Going to City Center: ', (10, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1, cv2.LINE_AA)
        cv2.putText(frame, str(car_counter), (500, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

        # Show the processed video frame
        cv2.imshow("Frame", frame)

        # Press 'q' to stop the video early
        if cv2.waitKey(1) == ord('q'):
            break

    # Print final car count and statistics
    print(f"Total cars going to city center: {car_counter}")
    print(f"Cars per minute: {round(car_counter / (video_duration / 60), 2)}")

    # Release video capture and close all OpenCV windows
    video_cap.release()
    cv2.destroyAllWindows()



In [10]:
# List of video files
video_files = ['Traffic_Laramie_1.mp4', 'Traffic_Laramie_2.mp4']

# Loop through both videos without modifying the main processing code
for video_path in video_files:
    video_cap = cv2.VideoCapture(video_path)  # Open the video

    # Initialize background subtractor (you can reuse it)
    bg_sub = cv2.createBackgroundSubtractorMOG2(history=15000, varThreshold=20, detectShadows=False)

    if not video_cap.isOpened():
        print(f"Failed to open {video_path}")
        continue  # Skip to the next video

    print(f"Processing video: {video_path}")

    # Run your existing car counting code here, unchanged
    # (Make sure your car_counter function is defined and used here)
    car_counter(video_cap, bg_sub)

    # Release video capture after processing
    video_cap.release()

cv2.destroyAllWindows()  # Close all OpenCV windows after processing both videos


Processing video: Traffic_Laramie_1.mp4
Total cars going to city center: 6
Cars per minute: 2.02
Processing video: Traffic_Laramie_2.mp4
Total cars going to city center: 6
Cars per minute: 3.41
